# Modelagem

## Tratamento (condensado)

Aqui é feito o tratamento completo, que foi melhor explicado em "treatment.ipynb"

Mudanças: 
1. Exclui mais colunas (ver ``# Exclusoes mais novas``)
2. Aglutina mais a coluna ``TP_ST_CONCLUSAO`` (ver ``# TP_CONCLUSAO``)
3. Aglutina coluna ``ESC_PAI`` e ``ESC_MAE``
4. Aglutina a coluna ``RENDA``
5. Aglutina a coluna ``REGIAO``

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Configurações
DATA_PATH = Path().resolve() / 'data'
ARQUIVO_AMOSTRA_PATH = DATA_PATH / 'raw' / 'microdados_enem_2023_sample.csv'

# Definição dos tipos
colunas_float = [
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
    'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
    'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'
]

colunas_string = [
    'NU_INSCRICAO', 'CO_MUNICIPIO_ESC', 'CO_MUNICIPIO_PROVA'
]

# Captura os nomes das colunas
colunas = pd.read_csv(ARQUIVO_AMOSTRA_PATH, nrows=0, encoding='latin1').columns.tolist()

# Preparar o dicionário de tipos
dtypes = {}
for col in colunas:
    if col in colunas_float:
        dtypes[col] = 'float32'
    elif col in colunas_string:
        dtypes[col] = 'string'
    else:
        dtypes[col] = 'category'

# Leitura com tipos otimizados
df = pd.read_csv(ARQUIVO_AMOSTRA_PATH, dtype=dtypes, encoding='latin1')

# REGIAO
# Define regioes...
df['REGIAO'] = df['CO_MUNICIPIO_PROVA'].apply(lambda x: x[0])
df['REGIAO'] = df['REGIAO'].astype('category')

#... e aglutina
condicoes = [
    (df['REGIAO'].isin(['1', '5'])),                 # N e C-O
    (df['REGIAO'].isin(['3', '4'])),                 # S e SE
    (df['REGIAO'].isin(['2'])),                     # NE
]

# Valores que serão atribuídos com base nas condições
valores = [1, 2, 3]

df['REGIAO'] =  np.select(condicoes, valores)
df['REGIAO'] = df['REGIAO'].astype('category')

df['REGIAO'] = df['REGIAO'].cat.rename_categories({
    '1.0': '1',
    '2.0': '2',
    '3.0': '3'
})

# Exclusoes
del df['NU_INSCRICAO']
del df['TP_FAIXA_ETARIA']
del df['CO_MUNICIPIO_ESC']                         
del df['TP_DEPENDENCIA_ADM_ESC']                   
del df['TP_LOCALIZACAO_ESC']                       
del df['TP_SIT_FUNC_ESC']
del df['TP_ENSINO']
del df['CO_MUNICIPIO_PROVA']

# TP_CONCLUSAO
condicoes = [
    (df['TP_ST_CONCLUSAO'] == '1'),                 # Concluido independente do
    (df['TP_ST_CONCLUSAO'].isin(['2', '3'])),       # Cursando EM
    (df['TP_ST_CONCLUSAO'] == '4')                  # EM não cursado
]

# Valores que serão atribuídos com base nas condições
valores = [1, 2, 3]

df['TP_CONCLUSAO'] =  np.select(condicoes, valores)
df['TP_CONCLUSAO'] = df['TP_CONCLUSAO'].astype('category')

df['TP_CONCLUSAO'] = df['TP_CONCLUSAO'].cat.rename_categories({
    '1.0': '1',
    '2.0': '2',
    '3.0': '3'
})

del df['TP_ANO_CONCLUIU']
del df['TP_ST_CONCLUSAO'] 

# TP_PRESENCA_DIA1 e TP_PRESENCA_DIA2
df.rename(columns={
    'TP_PRESENCA_CH': 'TP_PRESENCA_DIA1',
    'TP_PRESENCA_CN': 'TP_PRESENCA_DIA2'
}, inplace=True)

del df['TP_PRESENCA_LC']
del df['TP_PRESENCA_MT']

# CO_PROVA_DIA1
condicoes = [
    df['CO_PROVA_CH'].isin(['1191.0', '1192.0', '1193.0', '1194.0']),               # Prova Comum
    df['CO_PROVA_CH'].isin(['1195.0', '1196.0', '1197.0', '1198.0', '1199.0']),     # Prova Adaptada
    df['CO_PROVA_CH'].isin(['1271.0', '1272.0', '1273.0', '1274.0']),               # Prova Reaplicação
    df['CO_PROVA_CH'].isna()                                                        # Ausente
]

# Valores que serão atribuídos com base nas condições
valores = [1, 2, 3, 4]

df['CO_PROVA_DIA1'] =  np.select(condicoes, valores)
df['CO_PROVA_DIA1'] = df['CO_PROVA_DIA1'].astype('category')
del df['CO_PROVA_CH']
del df['CO_PROVA_LC'] 

# CO_PROVA_DIA2
condicoes = [
    df['CO_PROVA_CN'].isin(['1221.0', '1222.0', '1223.0', '1224.0']),               # Prova Comum
    df['CO_PROVA_CN'].isin(['1225.0', '1226.0', '1227.0', '1228.0', '1229.0']),     # Prova Adaptada
    df['CO_PROVA_CN'].isin(['1301.0', '1302.0', '1303.0', '1304.0']),               # Prova Reaplicação
    df['CO_PROVA_CN'].isna()                                            # Ausente
]

# Valores que serão atribuídos com base nas condições
valores = [1, 2, 3, 4]

df['CO_PROVA_DIA2'] = np.select(condicoes, valores)
df['CO_PROVA_DIA2'] = df['CO_PROVA_DIA2'].astype('category')
del df['CO_PROVA_CN']
del df['CO_PROVA_MT'] 

# ESC_MAE, ESC_PAI, INTERNET
df.rename(columns={
    'Q001': 'ESC_PAI',
    'Q002': 'ESC_MAE',
    'Q025': 'INTERNET'
}, inplace=True)

# ESC_PAI
condicoes = [
    df['ESC_PAI'].isin(['A']),                         # Nunca estudou
    df['ESC_PAI'].isin(['B', 'C', 'D']),               # Não tem EM completo
    df['ESC_PAI'].isin(['E', 'F', 'G']),               # Tem EM completo
    df['ESC_PAI'].isin(['H'])                         # Não sei
]

valores = [1, 2, 3, 4]

df['ESC_PAI'] =  np.select(condicoes, valores)
df['ESC_PAI'] = df['ESC_PAI'].astype('category')

# ESC_MAE
condicoes = [
    df['ESC_MAE'].isin(['A']),                         # Nunca estudou
    df['ESC_MAE'].isin(['B', 'C', 'D']),               # Não tem EM completo
    df['ESC_MAE'].isin(['E', 'F', 'G']),               # Tem EM completo
    df['ESC_MAE'].isin(['H'])                           # Não sei
]

valores = [1, 2, 3, 4]

df['ESC_MAE'] =  np.select(condicoes, valores)
df['ESC_MAE'] = df['ESC_MAE'].astype('category')

# RENDA
condicoes = [
    df['Q006'] == 'A',                  # Sem renda                    
    df['Q006'].isin(['B', 'C']),        # Renda até 1.5 SM
    (~df['Q006'].isin(['A', 'B', 'C']))   # Renda maior que 1.5 SM
]

valores = [1, 2, 3]

df['RENDA'] = np.select(condicoes, valores)
df['RENDA'] = df['RENDA'].astype(int).astype('category')

del df['Q006']

# TP_STATUS_REDACAO
condicoes = [
    df['TP_STATUS_REDACAO'] == '1.0',
    df['TP_STATUS_REDACAO'] == '2.0',
    df['TP_STATUS_REDACAO'] == '3.0',
    df['TP_STATUS_REDACAO'] == '4.0',
    df['TP_STATUS_REDACAO'] == '6.0',
    df['TP_STATUS_REDACAO'] == '7.0',
    df['TP_STATUS_REDACAO'] == '8.0',
    df['TP_STATUS_REDACAO'] == '9.0',
    df['TP_STATUS_REDACAO'].isna(),
]

valores = [1, 2, 3, 4, 6, 7, 8, 9, 10]

df['TP_STATUS_REDACAO'] =  np.select(condicoes, valores)
df['TP_STATUS_REDACAO'] = df['TP_STATUS_REDACAO'].astype('category')

# VARIÁVEIS NUMÉRICAS
# Transformacao das variaveis numericas em categoricas, segunda as faixas estipuladas
colunas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',  'NU_NOTA_REDACAO',
           'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']

for col in colunas:
    mediana = df[col].median()
    std = df[col].std()

    condicoes = [
        df[col].isna(),
        df[col] == 0,
        df[col] <= mediana,
        df[col] > mediana
    ]

    valores = [0, 1, 2, 3]

    df[col] =  np.select(condicoes, valores)
    df[col] = df[col].astype('category')

# # Proporcao das faixas das notas
# print("\n:")

# colunas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO',
#            'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']
# contagens = {}
# percentual = {}

# for col in colunas:
    
#     contagens[f'tp_{col}'] = df[col].value_counts(dropna=False)
#     percentual[f'tp_{col}_pct'] = df[col].value_counts(normalize=True, dropna=False) * 100

#     resultado = pd.DataFrame({
#         'Frequência': contagens[f'tp_{col}'],
#         'Percentual (%)': percentual[f'tp_{col}_pct'].round(2)
#     })
    
#     print(resultado, '\n')

# Exclusoes mais novas
del df['TP_NACIONALIDADE']
del df['TP_ESTADO_CIVIL']
del df['CO_PROVA_DIA1']
del df['CO_PROVA_DIA2']
del df['TP_STATUS_REDACAO']

# # Informações para conferência
# print("="*40)
# print(f"Memória usada: {df.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")
# print(f"Quantidade de colunas: {df.shape[1]}")
# print(f"Quantidade de linhas: {df.shape[0]}")
# print("="*40)

# # Tipos de dados
# print("\nTipos de dados por coluna:")
# print(df.dtypes)

In [ ]:
for col in df.select_dtypes(include='category').columns:
    valores = df[col].unique().tolist()
    try:
        # Tenta converter para número e ordenar
        valores_ordenados = sorted(valores, key=lambda x: float(x))
    except ValueError:
        # Se não forem numéricos, mantém a ordem original
        valores_ordenados = valores
    print(f'\n{col}: {valores_ordenados}')

df.info()

## Checa significâncias

In [ ]:
from utils import plot_barras_categoricas

desconsidera_cols = ['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']
plot_barras_categoricas(df.drop(columns=desconsidera_cols), min_sup=0.33)

## Aplicando o algoritmo (suporte = 0.33)

In [ ]:
# Transforma dataframe em formato transacional

desconsidera_cols = ['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']
df_binario = pd.get_dummies(df.drop(columns=desconsidera_cols), prefix_sep='_', dtype=bool)

print(f'Total de colunas: {len(df_binario.columns)}')
for col in df_binario.columns:
    print(col)

In [ ]:
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules
import time

# 1. Gerar itemsets frequentes usando FP-Growth
print('Gerando itemsets...')

start_time = time.time()
frequent_itemsets = fpgrowth(df_binario, min_support=0.33, use_colnames=True)
end_time = time.time()
print(f"\tTempo de execução: {end_time - start_time} segundos")

# 2. Visualizar os itemsets frequentes
print(frequent_itemsets)

# 3. Gerar regras de associação a partir dos itemsets
start_time = time.time()
regras = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
end_time = time.time()
print(f"\tTempo de execução: {end_time - start_time} segundos")

# 4. Visualizar as regras
print(regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

In [ ]:
# Exporta pra csv
RESULTS_PATH = Path().resolve() / 'results'
frequent_itemsets.to_csv(RESULTS_PATH / 'freq_itemsets_new_33.csv')
regras.to_csv(RESULTS_PATH / 'regras_new_33_conf50.csv')

## Analisando apenas quem foi os dois dias

Para focar a análise e encontrar padrões relevantes, decidimos focar a análise apenas nos candidatos que foram nos dois dias.

In [ ]:
from utils import plot_barras_categoricas

df_apenas_presentes = df[(df.TP_PRESENCA_DIA1 == '1') & (df.TP_PRESENCA_DIA2 == '1')].copy()
df_apenas_presentes.drop(columns=['TP_PRESENCA_DIA2', 'TP_PRESENCA_DIA1'], inplace=True)
print(f'Total anterior:\t {len(df)}')
print(f'Total novo:\t {len(df_apenas_presentes)}')

plot_barras_categoricas(df_apenas_presentes)

## Analisando apenas quem faltou pelo menos um dia

In [ ]:
from utils import plot_barras_categoricas

df_apenas_presentes = df[(df.TP_PRESENCA_DIA1 != '1') | (df.TP_PRESENCA_DIA2 != '1')].copy()
# df_apenas_presentes.drop(columns=['TP_PRESENCA_DIA2', 'TP_PRESENCA_DIA1'], inplace=True)
print(f'Total anterior:\t {len(df)}')
print(f'Total novo:\t {len(df_apenas_presentes)}')

plot_barras_categoricas(df_apenas_presentes)

## Apenas não treineiros

In [ ]:
from utils import plot_barras_categoricas

df_apenas_nao_treineiros = df[(df.IN_TREINEIRO == '0')].copy()
df_apenas_nao_treineiros.drop(columns=['IN_TREINEIRO'], inplace=True)
print(f'Total anterior:\t {len(df)}')
print(f'Total novo:\t {len(df_apenas_nao_treineiros)}')

plot_barras_categoricas(df_apenas_nao_treineiros)

## Apenas treineiros

In [ ]:
from utils import plot_barras_categoricas

df_apenas_treineiros = df[(df.IN_TREINEIRO == '1')].copy()
df_apenas_treineiros.drop(columns=['IN_TREINEIRO'], inplace=True)
print(f'Total anterior:\t {len(df)}')
print(f'Total novo:\t {len(df_apenas_treineiros)}')

plot_barras_categoricas(df_apenas_treineiros)